In [4]:
import time
import json
import random
from kafka import KafkaProducer
from datetime import datetime, timedelta

# Configuration
KAFKA_TOPIC = "forest-sensors"
KAFKA_SERVER = "kafka:29092"

print("Starting SEASONAL Weather Simulator...")

producer = KafkaProducer(
    bootstrap_servers=[KAFKA_SERVER],
    value_serializer=lambda x: json.dumps(x).encode('utf-8')
)

sensor_ids = [f"TR_SENS_{i:03d}" for i in range(1, 21)]
TRAINING_POINTS = [(38.0, 41.5), (38.0, 33.5)]

def get_location():
    base_lat, base_lon = random.choice(TRAINING_POINTS)
    return round(base_lat + random.uniform(-0.03, 0.03), 3), round(base_lon + random.uniform(-0.03, 0.03), 3)

def generate_seasonal_data():
    """
    Select a random season and generate appropriate data and timestamp for it.
    """

    season_choice = random.choices([0, 1, 2, 3], weights=[10, 20, 50, 20], k=1)[0]
    
    if season_choice == 0:  # WINTER (Months 12, 1, 2)
        month = random.choice([12, 1, 2])
        temp = random.uniform(0, 15)      # cold
        hum = random.uniform(60, 90)      # humid
        wind = random.uniform(5, 20)
        is_fire = False               
        season_name = "WINTER"

    elif season_choice == 1:  # SPRING (Months 3, 4, 5)
        month = random.choice([3, 4, 5])
        temp = random.uniform(15, 25)     # mild
        hum = random.uniform(40, 70)
        wind = random.uniform(5, 15)
        is_fire = random.random() < 0.05  # very small chance
        season_name = "SPRING"

    elif season_choice == 2:  # SUMMER (Months 6, 7, 8) - Danger Zone
        month = random.choice([6, 7, 8])
        # Allow fire-prone conditions here
        if random.random() < 0.4: # 40% heatwave
            temp = random.uniform(35, 48) # very hot
            hum = random.uniform(10, 30)  # very dry
            wind = random.uniform(20, 40)
            is_fire = True
            season_name = "SUMMER (HEATWAVE)"
        else:
            temp = random.uniform(25, 35)
            hum = random.uniform(30, 50)
            wind = random.uniform(10, 25)
            is_fire = False
            season_name = "SUMMER (Normal)"

    else:  # AUTUMN (Months 9, 10, 11)
        month = random.choice([9, 10, 11])
        temp = random.uniform(15, 28)
        hum = random.uniform(40, 60)
        wind = random.uniform(10, 30)
        is_fire = random.random() < 0.1   # small chance
        season_name = "AUTUMN"

    # --- Build the fake timestamp ---
    # Use year 2025 and change month/day to match the season
    year = 2025
    day = random.randint(1, 28)
    hour = random.randint(10, 16) # midday (peak heat)
    
    fake_timestamp = datetime(year, month, day, hour, random.randint(0, 59), random.randint(0, 59))
    timestamp_str = fake_timestamp.strftime("%Y-%m-%d %H:%M:%S")

    return timestamp_str, temp, hum, wind, season_name

try:
    msg_count = 0
    while True:
        msg_count += 1
        
        # 1. Get Data Logic
        ts, t, h, w, season = generate_seasonal_data()
        lat, lon = get_location()

        # 2. Build Message
        message = {
            "sensor_id": random.choice(sensor_ids),
            "timestamp": ts,  # send the fake timestamp
            "temp": round(t, 2),
            "humidity": round(h, 2),
            "wind": round(w, 2),
            "rain": 0.0,
            "coord_x": lat,
            "coord_y": lon,
            "battery": 90.0,
            "signal": 5
        }
        
        # 3. Send
        producer.send(KAFKA_TOPIC, message)
        
        print(f"Date: {ts} | Temp: {t:.1f}°C | Hum: {h:.1f}%")
        
        time.sleep(1) # sending rate

except KeyboardInterrupt:
    producer.close()

Starting SEASONAL Weather Simulator...
Date: 2025-03-24 14:48:12 | Temp: 23.4°C | Hum: 46.7%
Date: 2025-11-21 14:07:50 | Temp: 16.6°C | Hum: 51.5%
Date: 2025-07-23 15:59:13 | Temp: 45.6°C | Hum: 29.3%
Date: 2025-12-07 12:36:28 | Temp: 7.3°C | Hum: 83.6%
Date: 2025-08-23 15:13:53 | Temp: 31.2°C | Hum: 33.6%
Date: 2025-08-07 12:40:27 | Temp: 30.1°C | Hum: 30.9%
Date: 2025-09-11 15:07:53 | Temp: 18.4°C | Hum: 41.8%
Date: 2025-03-16 16:39:54 | Temp: 23.0°C | Hum: 54.0%
Date: 2025-04-10 14:05:44 | Temp: 17.2°C | Hum: 59.3%
Date: 2025-03-11 13:53:02 | Temp: 20.8°C | Hum: 42.2%
Date: 2025-02-23 11:18:57 | Temp: 7.9°C | Hum: 75.3%
Date: 2025-07-15 14:46:56 | Temp: 33.4°C | Hum: 44.3%
Date: 2025-07-17 15:29:14 | Temp: 28.2°C | Hum: 35.5%
Date: 2025-07-06 14:47:23 | Temp: 33.8°C | Hum: 32.2%
Date: 2025-06-26 16:27:03 | Temp: 39.2°C | Hum: 13.0%
Date: 2025-03-03 10:06:23 | Temp: 22.6°C | Hum: 54.3%
Date: 2025-07-15 15:38:00 | Temp: 25.9°C | Hum: 35.4%
Date: 2025-07-22 13:24:57 | Temp: 33.4°C | Hu